In [1]:
from datetime import datetime
import redis
import time
import pickle
import psycopg2
import json


In [3]:
# Connect to postgres database

connection = psycopg2.connect(
    host="localhost",
    database="testing",
    user="postgres",
    password="example"
)
postgres = connection.cursor()

In [ ]:
# Connect to redis

pool = redis.ConnectionPool(host="queue", port=6379, db=0)
r = redis.Redis(connection_pool=pool)
p = r.pubsub()

p.subscribe("Log")

p.get_message()
time.sleep(0.5)

In [ ]:
def getNextInQueue(p) :
    """ 
    Get incoming items in the queue as a generator
    """
    for mes in p.listen() :
        if type(mes["data"]) == type(b'') :
            yield pickle.loads(mes["data"])

In [ ]:
# SQL

def sqlGetUser(id) :
    """
    Get the sql query to create a new user
    """
    return "SELECT * FROM users WHERE users.extensionID = '{}';".format(id)

def sqlAddDiff(id, diff, date) :
    """
    Get the sql query to create a new diff
    """
    return "INSERT INTO diffs VALUES ({0}, '{3}', '{1}', '{2}')".format(id, json.dumps({"+": diff["+"]}), json.dumps({"-": diff["-"]}), date)

def sqlUpdateUser(id, newCurrent) :
    """
    Get the sql query to update the current lists for user with users.id = id  
    """
    return "UPDATE users SET current = '{0}' WHERE users.id = {1};".format(
            json.dumps({"current": newCurrent}),
            id
        )

def sqlCreateUser(id, current) :
    # return "INSERT INTO users(extensionID, current) VALUES ('{0}', '{1}')".format(id, json.dumps({"current": current}))
    return "INSERT INTO listoflists DEFAULT VALUES; \
    INSERT INTO users(extensionID, current) VALUES ('{}', currval('listoflists_id_seq'));".format(id)


In [13]:
def createUser(id, current) :
    postgres.execute("INSERT INTO listoflists DEFAULT VALUES RETURNING id;")
    listid = postgres.fetchone()[0]
    postgres.execute("INSERT INTO users(extensionID, current) VALUES ('{0}', {1});".format(id, listid))
    for l in current :
        postgres.execute("SELECT lists.id FROM lists WHERE lists.nom = '{}'".format(l))
        Lid = postgres.fetchone()[0]
        postgres.execute("INSERT INTO listoflists_lists VALUES ({0}, {1})".format(listid, Lid))
    connection.commit()

def updateUser(id, newCurrent) :
    postgres.execute("SELECT users.current FROM users WHERE users.id = {}".format(id))
    listid = postgres.fetchone()[0]
    postgres.execute("DELETE FROM listoflists_lists WHERE listoflists_lists.id = {}".format(listid))
    for l in newCurrent :
        postgres.execute("SELECT lists.id FROM lists WHERE lists.nom = '{}'".format(l))
        Lid = postgres.fetchone()[0]
        postgres.execute("INSERT INTO listoflists_lists VALUES ({0}, {1})".format(listid, Lid))
    
    connection.commit()

def addDiff(id, diff, date) :
    postgres.execute("INSERT INTO listoflists DEFAULT VALUES RETURNING id;")
    plusid = postgres.fetchone()[0]
    postgres.execute("INSERT INTO listoflists DEFAULT VALUES RETURNING id;")
    minusid = postgres.fetchone()[0]
    postgres.execute("INSERT INTO diffs VALUES ({0}, '{3}', {1}, {2})".format(id, plusid, minusid, date))
    for l in diff["+"] :
        postgres.execute("SELECT lists.id FROM lists WHERE lists.nom = '{}'".format(l))
        Lid = postgres.fetchone()[0]
        postgres.execute("INSERT INTO listoflists_lists VALUES ({0}, {1})".format(plusid, Lid))
    for l in diff["-"] :
        postgres.execute("SELECT lists.id FROM lists WHERE lists.nom = '{}'".format(l))
        Lid = postgres.fetchone()[0]
        postgres.execute("INSERT INTO listoflists_lists VALUES ({0}, {1})".format(minusid, Lid))
    connection.commit()

connection.commit()
# updateUser(3, ["L1"])
addDiff(3, {"+": [], "-":["L2"]}, str(datetime.now()))

In [ ]:
for item in getNextInQueue(p) :
    # Get corresponding user
    postgres.execute(sqlGetUser(item[0]))
    user = postgres.fetchone()
    date = str(datetime.now())

    if user != None :
        # user already exists
        if user[-1]["current"] == item[1] :
            continue

        diff = {
            "+" : [l for l in item[1] if l not in user[-1]["current"]],
            "-" : [l for l in user[-1]["current"] if l not in item[1]]
        }

        postgres.execute(sqlAddDiff(user[0], diff, date))
        postgres.execute(sqlUpdateUser(user[0], item[1]))
    else :
        # create new user
        postgres.execute(sqlCreateUser(item[0], item[1]))
        postgres.execute(sqlGetUser(item[0]))
        user = postgres.fetchone()
        # Add initial diff
        postgres.execute(sqlAddDiff(user[0], {"+": item[1], "-": []}, date))

    # Commit changes to the database
    connection.commit()


    

        